In [1]:
using Rocket
using MacroTools

┌ Info: Precompiling Rocket [df971d30-c9d6-4b37-b8ff-e965b2cb3a40]
└ @ Base loading.jl:1278


In [11]:
l = lazy(Int)

LazyObservable(Int64)

In [16]:
sub1 = subscribe!(l, logger())

[LogActor] Data: 1
[LogActor] Completed


Rocket.LazyObservableSubscription(nothing, Rocket.LazyObservableSubscriptionProps(VoidTeardown()))

In [15]:
set!(l, of(1));

In [16]:
__extract_interfaces_macro_rule(:([
    out,
    (μ, aliases = [ mean ]),
    (v, aliases = [ var ])
]))

:([out, (μ, aliases = [mean]), (v, aliases = [var])])

In [14]:
unsubscribe!(sub1)